In [15]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from meteostat import Point, Daily
from geopy.geocoders import Nominatim
from datetime import timedelta

In [16]:
# --- Load Dataset ---
# Import the dataset uploaded to the repo
url = "https://raw.githubusercontent.com/MaharLeika18/Data-Mining---Python/refs/heads/main/Final_Exam/Raw%20Data.csv"
data = pd.read_csv(url)

print("Data loaded successfully!")
print(data.head())

Data loaded successfully!
   Order Date  Customer Name         State         Category Sub-Category  \
0  03-01-2014  Darren Powers         Texas  Office Supplies        Paper   
1  04-01-2014  Phillina Ober      Illinois  Office Supplies       Labels   
2  04-01-2014  Phillina Ober      Illinois  Office Supplies      Storage   
3  04-01-2014  Phillina Ober      Illinois  Office Supplies      Binders   
4  05-01-2014     Mick Brown  Pennsylvania  Office Supplies          Art   

                                        Product Name   Sales  Quantity  Profit  
0  Message Book, Wirebound, Four 5 1/2" X 4" Form...   16.45         2    5.55  
1                                          Avery 508   11.78         3    4.27  
2                      SAFCO Boltless Steel Shelving  272.74         3  -64.77  
3         GBC Standard Plastic Binding Systems Combs    3.54         2   -5.49  
4  Avery Hi-Liter EverBold Pen Style Fluorescent ...   19.54         3    4.88  


In [17]:
# --- Data Preprocessing ---
# Create id column
data['Transaction ID'] = data.index + 1

# Convert dates to datetime
data['Order Date'] = pd.to_datetime(data['Order Date'], format='%d-%m-%Y')
start = data['Order Date'].min()
end = data['Order Date'].max()
print(f"Date range: {start.date()} to {end.date()}")

# Convert states to coordinates
# LOUE NOTE: FULL DISCLAIMER I had chatgpt gen these coords cuz no way am i doing this manually.
# so if the cords are innacurate im sorry
# or well chatgpt should be sorry but unlike caine its not sentient

state_coords = {
    "Alabama": (32.806671, -86.791130),
    "Alaska": (61.370716, -152.404419),
    "Arizona": (33.729759, -111.431221),
    "Arkansas": (34.969704, -92.373123),
    "California": (36.116203, -119.681564),
    "Colorado": (39.059811, -105.311104),
    "Connecticut": (41.597782, -72.755371),
    "Delaware": (39.318523, -75.507141),
    "Florida": (27.766279, -81.686783),
    "Georgia": (33.040619, -83.643074),
    "Hawaii": (21.094318, -157.498337),
    "Idaho": (44.240459, -114.478828),
    "Illinois": (40.349457, -88.986137),
    "Indiana": (39.849426, -86.258278),
    "Iowa": (42.011539, -93.210526),
    "Kansas": (38.526600, -96.726486),
    "Kentucky": (37.668140, -84.670067),
    "Louisiana": (31.169546, -91.867805),
    "Maine": (44.693947, -69.381927),
    "Maryland": (39.063946, -76.802101),
    "Massachusetts": (42.230171, -71.530106),
    "Michigan": (43.326618, -84.536095),
    "Minnesota": (45.694454, -93.900192),
    "Mississippi": (32.741646, -89.678696),
    "Missouri": (38.456085, -92.288368),
    "Montana": (46.921925, -110.454353),
    "Nebraska": (41.125370, -98.268082),
    "Nevada": (38.313515, -117.055374),
    "New Hampshire": (43.452492, -71.563896),
    "New Jersey": (40.298904, -74.521011),
    "New Mexico": (34.840515, -106.248482),
    "New York": (42.165726, -74.948051),
    "North Carolina": (35.630066, -79.806419),
    "North Dakota": (47.528912, -99.784012),
    "Ohio": (40.388783, -82.764915),
    "Oklahoma": (35.565342, -96.928917),
    "Oregon": (44.572021, -122.070938),
    "Pennsylvania": (40.590752, -77.209755),
    "Rhode Island": (41.680893, -71.511780),
    "South Carolina": (33.856892, -80.945007),
    "South Dakota": (44.299782, -99.438828),
    "Tennessee": (35.747845, -86.692345),
    "Texas": (31.054487, -97.563461),
    "Utah": (40.150032, -111.862434),
    "Vermont": (44.045876, -72.710686),
    "Virginia": (37.769337, -78.169968),
    "Washington": (47.400902, -121.490494),
    "West Virginia": (38.491226, -80.954456),
    "Wisconsin": (44.268543, -89.616508),
    "Wyoming": (42.755966, -107.302490)
}

data['Coords'] = data['State'].map(state_coords)
data[['Latitude', 'Longitude']] = pd.DataFrame(data['Coords'].tolist(), index=data.index)

data.head()

Date range: 2014-01-03 to 2017-12-30


,Order Date,Customer Name,State,Category,Sub-Category,Product Name,Sales,Quantity,Profit,Transaction ID,Coords,Latitude,Longitude
0,2014-01-03,Darren Powers,Texas,Office Supplies,Paper,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",16.45,2,5.55,1,"(31.054487, -97.563461)",31.054487,-97.563461
1,2014-01-04,Phillina Ober,Illinois,Office Supplies,Labels,Avery 508,11.78,3,4.27,2,"(40.349457, -88.986137)",40.349457,-88.986137
2,2014-01-04,Phillina Ober,Illinois,Office Supplies,Storage,SAFCO Boltless Steel Shelving,272.74,3,-64.77,3,"(40.349457, -88.986137)",40.349457,-88.986137
3,2014-01-04,Phillina Ober,Illinois,Office Supplies,Binders,GBC Standard Plastic Binding Systems Combs,3.54,2,-5.49,4,"(40.349457, -88.986137)",40.349457,-88.986137
4,2014-01-05,Mick Brown,Pennsylvania,Office Supplies,Art,Avery Hi-Liter EverBold Pen Style Fluorescent ...,19.54,3,4.88,5,"(40.590752, -77.209755)",40.590752,-77.209755


In [ ]:
# Get historical weather data for each state
weather_records = []

# Get unique states with their coordinates
unique_states = data[['State', 'Latitude', 'Longitude']].drop_duplicates()

print(f"Getting weather data for {len(unique_states)} unique states...")

states_with_data = 0
for i, row in unique_states.iterrows():
    state = row['State']
    lat, lon = row['Latitude'], row['Longitude']
    location = Point(lat, lon)

    try:
        w = Daily(location, start, end).fetch()
        if not w.empty:
            w = w.reset_index()
            w['State'] = state
            w['Latitude'] = lat
            w['Longitude'] = lon
            weather_records.append(w)
            print(f"Weather data retrieved for {state}")
            states_with_data += 1
    except Exception as e:
        print(f"Failed to fetch weather for {state}: {e}")
        continue

if weather_records:
    weather_data = pd.concat(weather_records, ignore_index=True)
    print(f"\n Using state weather data for {states_with_data}/{len(unique_states)} states")
#If no coords for state (Mary suggestion: use USA center?)
else:
    print("\nNo state weather data retrieved. Using central US fallback...")
    central_location = Point(39.8283, -98.5795)  # Geographic center of US (Kansas according to Wikipedia)
    
    try:
        weather_data = Daily(central_location, start, end).fetch()
        if not weather_data.empty:
            weather_data = weather_data.reset_index()
            print("✓ Using central US weather data for all states")
        else:
            print("No weather data available")
            weather_data = pd.DataFrame()
    except Exception as e:
        print(f"Error: {e}")
        weather_data = pd.DataFrame()

# Display
print(f"\nWeather data shape: {weather_data.shape}")
display(weather_data.head())

Getting weather data for 49 unique states...
✓ Weather data retrieved for Texas
✓ Weather data retrieved for Illinois
✓ Weather data retrieved for California
✓ Weather data retrieved for Kentucky
✓ Weather data retrieved for Virginia
✓ Weather data retrieved for South Carolina
✓ Weather data retrieved for Ohio
✓ Weather data retrieved for Arkansas
✓ Weather data retrieved for Michigan
✓ Weather data retrieved for Tennessee
✓ Weather data retrieved for Florida
✓ Weather data retrieved for Nevada
✓ Weather data retrieved for Indiana
✓ Weather data retrieved for New York
✓ Weather data retrieved for Wisconsin
✓ Weather data retrieved for New Jersey
✓ Weather data retrieved for Missouri
✓ Weather data retrieved for North Carolina
✓ Weather data retrieved for Utah
✓ Weather data retrieved for Minnesota
✓ Weather data retrieved for Iowa
✓ Weather data retrieved for Massachusetts
✓ Weather data retrieved for Maryland
✓ Weather data retrieved for Connecticut
✓ Weather data retrieved for New Ha

,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,State,Latitude,Longitude
0,2014-01-03,3.7,-5.0,13.0,0.0,<NA>,<NA>,11.4,<NA>,1030.0,<NA>,Texas,31.054487,-97.563461
1,2014-01-04,12.9,5.0,22.0,0.0,<NA>,<NA>,20.3,<NA>,1015.9,<NA>,Texas,31.054487,-97.563461
2,2014-01-05,5.0,-1.0,14.0,0.0,<NA>,<NA>,25.2,<NA>,1023.1,<NA>,Texas,31.054487,-97.563461
3,2014-01-06,-4.1,-8.0,0.0,0.0,<NA>,<NA>,12.9,<NA>,1041.3,<NA>,Texas,31.054487,-97.563461
4,2014-01-07,0.7,-7.0,8.0,0.0,<NA>,<NA>,13.7,<NA>,1035.6,<NA>,Texas,31.054487,-97.563461


In [19]:
# Add description to the data
def get_weather_description(row):
    temp = row['tavg']  # average temperature
    precip = row['prcp']  # precipitation
    
    if pd.isna(temp) or pd.isna(precip):
        return "Unknown"
    
    if precip > 10: 
        return "Heavy Rain"
    elif precip > 5:
        return "Rainy"
    elif precip > 1:
        return "Light Rain" 
    elif temp > 29:
        return "Sunny"
    elif temp > 27:
        return "Partly Cloudy"
    else: 
        return "Warm and Humid"

# Apply weather descriptions
weather_data['weather'] = weather_data.apply(get_weather_description, axis=1)

print("Weather text descriptions:")
print(weather_data['weather'].value_counts())
weather_data[['tavg', 'prcp', 'weather']].head(10)

Weather text descriptions:
weather
Unknown           21308
Warm and Humid    17415
Light Rain         1659
Heavy Rain         1481
Rainy               852
Partly Cloudy       624
Sunny               365
Name: count, dtype: int64


,tavg,prcp,weather
0,3.7,0.0,Warm and Humid
1,12.9,0.0,Warm and Humid
2,5.0,0.0,Warm and Humid
3,-4.1,0.0,Warm and Humid
4,0.7,0.0,Warm and Humid
5,7.6,0.9,Warm and Humid
6,12.8,0.3,Warm and Humid
7,18.2,0.0,Warm and Humid
8,12.5,0.0,Warm and Humid
9,13.6,0.0,Warm and Humid


In [21]:
# Create weather_data_reset for merging
weather_data_reset = weather_data.reset_index()
weather_data_reset.rename(columns={'time': 'Date'}, inplace=True)

print(f"Shape: {weather_data_reset.shape}")
display(weather_data_reset[['Date', 'weather']].head())

# Rename your data column to match
data_merged = data.rename(columns={'Order Date': 'Date'})
data_with_weather = pd.merge(data_merged, weather_data_reset[['Date','weather','tavg','prcp','wspd']], on='Date', how='left')

Shape: (43704, 16)


,Date,weather
0,2014-01-03,Warm and Humid
1,2014-01-04,Warm and Humid
2,2014-01-05,Warm and Humid
3,2014-01-06,Warm and Humid
4,2014-01-07,Warm and Humid


In [25]:
# Export
data_with_weather.to_csv('retail_data_with_weather.csv', index=False)
display(data_with_weather)

,Date,Customer Name,State,Category,Sub-Category,Product Name,Sales,Quantity,Profit,Transaction ID,Coords,Latitude,Longitude,weather,tavg,prcp,wspd
0,2014-01-03,Darren Powers,Texas,Office Supplies,Paper,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",16.45,2,5.55,1,"(31.054487, -97.563461)",31.054487,-97.563461,Warm and Humid,3.7,0.0,11.4
1,2014-01-03,Darren Powers,Texas,Office Supplies,Paper,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",16.45,2,5.55,1,"(31.054487, -97.563461)",31.054487,-97.563461,Unknown,-8.0,<NA>,26.7
2,2014-01-03,Darren Powers,Texas,Office Supplies,Paper,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",16.45,2,5.55,1,"(31.054487, -97.563461)",31.054487,-97.563461,Warm and Humid,7.6,0.0,2.6
3,2014-01-03,Darren Powers,Texas,Office Supplies,Paper,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",16.45,2,5.55,1,"(31.054487, -97.563461)",31.054487,-97.563461,Unknown,-10.2,<NA>,6.2
4,2014-01-03,Darren Powers,Texas,Office Supplies,Paper,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",16.45,2,5.55,1,"(31.054487, -97.563461)",31.054487,-97.563461,Unknown,-4.5,<NA>,11.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299723,2017-12-30,Jill Matthias,Colorado,Office Supplies,Fasteners,Bagged Rubber Bands,3.02,3,-0.60,9994,"(39.059811, -105.311104)",39.059811,-105.311104,Unknown,-3.8,<NA>,13.5
299724,2017-12-30,Jill Matthias,Colorado,Office Supplies,Fasteners,Bagged Rubber Bands,3.02,3,-0.60,9994,"(39.059811, -105.311104)",39.059811,-105.311104,Warm and Humid,-15.4,0.0,18.4
299725,2017-12-30,Jill Matthias,Colorado,Office Supplies,Fasteners,Bagged Rubber Bands,3.02,3,-0.60,9994,"(39.059811, -105.311104)",39.059811,-105.311104,Unknown,-19.9,<NA>,1.1
299726,2017-12-30,Jill Matthias,Colorado,Office Supplies,Fasteners,Bagged Rubber Bands,3.02,3,-0.60,9994,"(39.059811, -105.311104)",39.059811,-105.311104,Light Rain,-10.3,1.5,4.7


In [26]:
# --- One-hot encode categorical data ---
# LOUE NOTE: Do this immediately before tha association analysis
# Or merge df with data because df contains only the one-hot encoded data

transactions = data.groupby('Transaction ID')['Category'].apply(list).values.tolist()
transactions = data.groupby('Transaction ID')['Sub-Category'].apply(list).values.tolist()

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

df.head()

,Accessories,Appliances,Art,Binders,Bookcases,Chairs,Copiers,Envelopes,Fasteners,Furnishings,Labels,Machines,Paper,Phones,Storage,Supplies,Tables
0,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
3,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
